In [3]:
%load_ext autoreload
%autoreload 2
import utils; utils.setup()

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns

import visualization
import data

In [5]:
def perc(n, m = None):
    p = n / m if m else n
    p = p * 1000
    p = int(p)
    p = p / 10
    return p


In [6]:
from pathlib import Path
project_root = Path().resolve().parent

intervals = {
    '15m': '15min',
    '4h':  '4H',
    '1h':  'H',
    '1d':  'D',
}



In [8]:
raw_interval = '15m'
interval = '1h'

raw_data_1min = data.load_symbol('solusdt', dir=project_root, interval=raw_interval)


In [9]:
df = data.set_interval(raw_data_1min, intervals[interval])
samples = df.shape[0]

/home/jbc/Work/indi/matplotlib-visualization-lib/src/data/__init__.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  grouper = df.groupby(pd.Grouper(level='timestamp_nanos', freq=freq))


In [11]:
periods = 8

close = df['close'].to_numpy()[:-periods]
close = close * np.ones((periods, len(close)))
close = close.T

rolling = {
    'high': (pd.DataFrame([window.to_list() for window in df['high'].rolling(periods)], index=df.index).dropna()[1:] - close) / close,
    'low':  (pd.DataFrame([window.to_list() for window in df['low'] .rolling(periods)], index=df.index).dropna()[1:] - close) / close,
}

In [70]:
gain_threshold = .02
poi = (rolling['high'] > gain_threshold).any(axis=1)
poi

timestamp_nanos
2023-03-24 22:00:00    False
2023-03-24 23:00:00    False
2023-03-25 00:00:00    False
2023-03-25 01:00:00    False
2023-03-25 02:00:00    False
                       ...  
2023-12-31 19:00:00    False
2023-12-31 20:00:00    False
2023-12-31 21:00:00    False
2023-12-31 22:00:00    False
2023-12-31 23:00:00    False
Freq: h, Length: 6770, dtype: bool

In [81]:
from dataclasses import dataclass

@dataclass
class Prediction:
    date: np.datetime64
    state: str
    hit: bool

    def value(self):
        return self.hit if self.state == 'valid' else False
    
    def is_incomplete(self):
        return self.state != 'incomplete'

def indicator(df, date: np.datetime64):
    hit = poi.loc[date]
    #hit = True
    return Prediction(date=date, state='valid', hit=hit)

def prediction(df, indicator):
    df = df.iloc[periods:]
    predictions = [ indicator(df, index ) for index in df.index ]
    return pd.Series([ p.value() for p in predictions ], index=df.index)

goi = prediction(df, indicator=indicator)

def confusion_matrix(poi, goi):
    poi = poi.to_numpy()
    goi = goi.to_numpy()
    out = pd.DataFrame([(poi & goi), (poi & np.invert(goi)), (np.invert(poi) & goi), (np.invert(poi) & np.invert(goi))]).T
    out.columns=['TP', 'FP', 'FN', 'TN']
    out = out.sum()
    return out
confusion_matrix(poi, goi)


TP    1508
FP       0
FN       0
TN    5262
dtype: int64